In [1]:
import pickle as p
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/",one_hot=False)

def strip_first_col(fname, delimiter=None):
    with open(fname, 'rb') as fin:
        for line in fin:
            try:
               yield line.split(delimiter, 1)[1]
            except IndexError:
               continue

sess = tf.Session()
# test the session device
a = tf.constant([1.0, 2.0])
b = tf.constant([3.0, 4.0])
c = a * b

# Launch the graph in a session.
sess = tf.Session()

# Evaluate the tensor 'c'.
print sess.run(c)
sess.close()

data_status = 0
raw_data = []
if data_status == 0:
    global raw_data
    raw_data = np.load("./raw_data.npy")
elif data_status == 1: 
    raw_data = np.loadtxt(strip_first_col('microarray.original.txt'),delimiter="\t",skiprows=1)
    np.save("raw_data", raw_data)
    data_status = 1

    # tuple to list
    raw_data = map(lambda x:list(x), raw_data)
    raw_data = np.array(raw_data)
    raw_data = np.transpose(raw_data)
    raw_data.shape
    train_value = arg = tf.convert_to_tensor(raw_data, dtype=tf.float32)
    train_value.shape
    data = tf.Variable(train_value, name="traindata")
print "Data loaded", raw_data.shape

def get_batch(data, batch_size, capacity):
    data = tf.cast(data, tf.float32)
    input_queue = tf.train.slice_input_producer([data])
    data = input_queue[0]
    data_batch = tf.train.batch([data], batch_size=batch_size, num_threads=8, 
                                    capacity=capacity,allow_smaller_final_batch=False)
    return data_batch


[ 3.  8.]
Data loaded (5896, 22283)


In [2]:
CAPACITY = 30000
training_epochs = 10
learning_rate = 0.01
display_step = 1
examples_to_show = 10
n_input = 22283 #Need to change to 22283

# tf Graph input (only pictures)
X = tf.placeholder("float",[None,n_input])
# X = get_batch(data,BATCH_SIZE,CAPACITY)

#need to change to 4 layers 
n_hidden_1 = 10000
n_hidden_2 = 5000
n_hidden_3 = 1250
n_hidden_4 = 300

weights= {
    'encoder_h1': tf.get_variable('eh1', [n_input,n_hidden_1],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
    'encoder_h2': tf.get_variable('eh2', [n_hidden_1,n_hidden_2],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
    'encoder_h3': tf.get_variable('eh3', [n_hidden_2,n_hidden_3],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
    'encoder_h4': tf.get_variable('eh4', [n_hidden_3,n_hidden_4],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h1': tf.get_variable('dh1', [n_hidden_4,n_hidden_3],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h2': tf.get_variable('dh2', [n_hidden_3,n_hidden_2],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h3': tf.get_variable('dh3', [n_hidden_2,n_hidden_1],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
    'decoder_h4': tf.get_variable('dh4', [n_hidden_1,n_input],tf.float32, initializer=tf.contrib.layers.xavier_initializer()),
#     'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1)),
#     'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3], stddev=0.1)),
#     'encoder_h4': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4], stddev=0.1)),
#     'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_4,n_hidden_3], stddev=0.1)),
#     'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_2], stddev=0.1)),
#     'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_1], stddev=0.1)),
#     'decoder_h4': tf.Variable(tf.truncated_normal([n_hidden_1,n_input], stddev=0.1)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([n_input])),
}
# weightse1_summary = tf.summary.histogram("weights_e1", weights['encoder_h1'])
# weightse2_summary = tf.summary.histogram("weights_e2", weights['encoder_h2'])
# weightse3_summary = tf.summary.histogram("weights_e3", weights['encoder_h3'])
# weightsd1_summary = tf.summary.histogram("weights_d1", weights['decoder_h1'])
# weightsd2_summary = tf.summary.histogram("weights_d2", weights['decoder_h1'])
# weightsd3_summary = tf.summary.histogram("weights_d3", weights['decoder_h1'])

def encoder(x):  
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['encoder_h1']),  
                                   biases['encoder_b1']))  
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['encoder_h2']),  
                                   biases['encoder_b2']))  
    layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.tanh(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    return layer_4

def decoder(x):  
    layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['decoder_h1']),  
                                   biases['decoder_b1']))  
    layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['decoder_h2']),  
                                   biases['decoder_b2']))  
    layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.tanh(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op
y_true = X

In [3]:
# cost = tf.reduce_mean(tf.pow(y_true-y_pred,2))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
cost_summary = tf.summary.scalar('Cost', cost)  
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:  
    init = tf.initialize_all_variables()  
else:  
    init = tf.global_variables_initializer()  
sess.run(init)  

In [7]:
index = np.random.permutation(raw_data.shape[0])

In [8]:
def get_batch_np(itert, batchsize):
    batch_index = index[itert*batchsize:(itert+1)*batchsize]
    batch_data = raw_data[batch_index]
    return batch_data

In [9]:
BATCH_SIZE = 64
learning_rate = 0.001
training_epochs = 5
# 首先计算总批数，保证每次循环训练集中的每个样本都参与训练，不同于批量训练  
total_batch = int(raw_data.shape[0]/BATCH_SIZE) #总批数  

merged_summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter('./tmp/logs', sess.graph)
total_step = 0

for epoch in range(training_epochs):  
    index = np.random.permutation(raw_data.shape[0])
    for i in range(total_batch):  
        total_step += 1
        batch_xs= get_batch_np(i, BATCH_SIZE)  # max(x) = 1, min(x) = 0  
        
        # Run optimization op (backprop) and cost op (to get loss value)  
        _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        
        
#         summary_str = sess.run(merged_summary_op, feed_dict={X: batch_xs})
#         summary_writer.add_summary(summary_str, total_step)
#         summary_writer.add_graph(sess.graph)
        
        if i % 10 == 0 or i<10:
            print "Iter: %d" % i, "cost =", "{:.9f}".format(c) 
    if epoch % display_step == 0:  
        record = open("train_loss.txt", "a+")
        record.write("epoch:"+str(epoch)+" loss:"+"{:.9f}\n".format(c))
        record.close()
        print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c)  
        init_op = tf.global_variables_initializer()
        saver = tf.train.Saver()
        with tf.Session() as sesstemp:
            sesstemp.run(init_op)
            saver_path = saver.save(sesstemp, "model.ckpt")
            print "Model saved in file: ", saver_path
print("Optimization Finished!")  

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python2.7/dist-pac

In [1]:
6+6

12